In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#stuff for LDA topic modeling of tweets

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

/Users/Daphne/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/Users/Daphne/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
/Users/Daphne/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://nump

In [5]:
df = pd.read_csv('vader_bmore.csv')

In [6]:
def clean_text(text):  
    pat1 = r'@[^ ]+'                   
    pat2 = r'https?://[A-Za-z0-9./]+'  
    pat3 = r'\'s'                      
    pat4 = r'\#\w+'                     
    pat5 = r'&amp '                     
    pat6 = r'[^A-Za-z\s]'               
    combined_pat = r'|'.join((pat1, pat2,pat3,pat4,pat5, pat6))
    text = re.sub(combined_pat,"",text).lower()
    return text.strip()

In [7]:
df['text'] = df['text'].apply(clean_text)

In [8]:
#Testing lemmatizer
print(WordNetLemmatizer().lemmatize('went', pos = 'v'))

go


In [9]:
#Testing stemmer
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [10]:
#text preprocess
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [11]:
# Testing preprocessing function

doc_sample = 'This disk has failed many times. I would like to get it replaced.'

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['This', 'disk', 'has', 'failed', 'many', 'times.', 'I', 'would', 'like', 'to', 'get', 'it', 'replaced.']


Tokenized and lemmatized document: 
['disk', 'fail', 'time', 'like', 'replac']


In [12]:
text_clean = [] 

for doc in df.text:
    text_clean.append(preprocess(doc))

#Create dictionary of lemmatized words
dictionary = gensim.corpora.Dictionary(text_clean)

In [13]:
#Checking dictionary created

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 baltimorean
1 eternal
2 forev
3 francisco
4 pelosi
5 peopl
6 repres
7 speaker
8 appropri
9 barrier
10 field


In [14]:
#Create bag of words from dictionary
twit_bag = [dictionary.doc2bow(doc) for doc in text_clean]

In [15]:
twit_bag[0]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]

In [16]:
#shows bag of words for first entry in dataset
bow_doc_x = twit_bag[0]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

Word 0 ("baltimorean") appears 1 time.
Word 1 ("eternal") appears 1 time.
Word 2 ("forev") appears 1 time.
Word 3 ("francisco") appears 1 time.
Word 4 ("pelosi") appears 1 time.
Word 5 ("peopl") appears 1 time.
Word 6 ("repres") appears 1 time.
Word 7 ("speaker") appears 1 time.


In [80]:
tweets_lda = gensim.models.LdaMulticore(twit_bag, 
                                   num_topics = 15, 
                                   id2word = dictionary,                                    
                                   passes = 20,
                                   random_state = 1,
                                   workers = 2)

In [81]:
for idx, topic in tweets_lda.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.030*"water" + 0.026*"area" + 0.021*"test" + 0.020*"weather" + 0.018*"special" + 0.017*"point" + 0.017*"expect" + 0.014*"storm" + 0.014*"latest" + 0.014*"even"


Topic: 1 
Words: 0.036*"covid" + 0.025*"maryland" + 0.022*"case" + 0.021*"report" + 0.019*"death" + 0.016*"vaccin" + 0.014*"number" + 0.014*"ann" + 0.014*"arundel" + 0.013*"believ"


Topic: 2 
Words: 0.092*"citi" + 0.076*"baltimor" + 0.023*"public" + 0.017*"mayor" + 0.016*"council" + 0.016*"hear" + 0.015*"court" + 0.015*"meet" + 0.013*"state" + 0.013*"announc"


Topic: 3 
Words: 0.080*"polic" + 0.044*"baltimor" + 0.042*"shoot" + 0.029*"offic" + 0.025*"yearold" + 0.022*"counti" + 0.020*"say" + 0.019*"charg" + 0.019*"kill" + 0.019*"investig"


Topic: 4 
Words: 0.066*"maryland" + 0.033*"elect" + 0.027*"state" + 0.027*"vote" + 0.027*"beat" + 0.015*"counti" + 0.014*"democrat" + 0.013*"candid" + 0.012*"race" + 0.012*"ballot"


Topic: 5 
Words: 0.040*"live" + 0.035*"raven" + 0.028*"news" + 0.027*"talk" + 0.025*"balt

In [82]:
df['main_topic'] = [int(str(sorted(tweets_lda[i],reverse=True,key=lambda x: x[1])[0][0]).zfill(3)) for i in twit_bag]

In [83]:
df.head()

,Unnamed: 0,Unnamed: 0.1,date,author,twitter_name,text,number_of_likes,number_of_retweets,cleaned_tweet,Compound,Positive,Negative,Neutral,Sentiment,main_topic
0,0,0.0,2022-11-18 02:52:03+00:00,Brandon M. Scott,MayorBMScott,speaker pelosi may represent the people of san...,36,4,speaker pelosi may represent the people of san...,0.0000,0.000,0.0,1.000,negative,12
1,1,1.0,2022-11-18 02:52:02+00:00,Brandon M. Scott,MayorBMScott,so many of my peers can thank her for knocking...,26,2,so many of my peers can thank her for knocking...,0.5106,0.185,0.0,0.815,neutral,0
2,2,2.0,2022-11-18 02:52:02+00:00,Brandon M. Scott,MayorBMScott,in addition to being the first female speaker ...,9,0,in addition to being the first female speaker ...,0.4754,0.140,0.0,0.860,neutral,10
3,3,3.0,2022-11-18 02:52:01+00:00,Brandon M. Scott,MayorBMScott,it brings us all great pride that a daughter o...,11,2,it brings us all great pride that a daughter o...,0.7579,0.245,0.0,0.755,positive,9
4,4,4.0,2022-11-18 02:52:00+00:00,Brandon M. Scott,MayorBMScott,on behalf of the city of baltimore i want to c...,221,12,on behalf of the city of baltimore i want to c...,0.5423,0.220,0.0,0.780,neutral,10


In [89]:
df.to_csv('lda_VADERbmore.csv')

In [26]:
import pyLDAvis

In [32]:
import pyLDAvis.gensim_models

In [47]:
pyLDAvis.enable_notebook()

In [84]:
vis = pyLDAvis.gensim_models.prepare(tweets_lda, twit_bag, dictionary=dictionary)

/Users/Daphne/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [85]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
11     0.138086 -0.040045       1        1  9.704824
2      0.105905  0.158680       2        1  9.658341
14     0.156815  0.124661       3        1  8.319426
5      0.132023  0.055063       4        1  7.258843
3      0.033563  0.182674       5        1  7.050100
9      0.101530 -0.193856       6        1  6.888427
0     -0.083928  0.116186       7        1  6.714798
8      0.038662 -0.056243       8        1  6.516495
4     -0.012624  0.122243       9        1  6.433569
13    -0.014798 -0.264618      10        1  6.352952
12     0.140350 -0.155567      11        1  6.323694
1     -0.088941  0.024621      12        1  5.890158
10    -0.122975 -0.058177      13        1  5.393659
7     -0.228337 -0.040813      14        1  3.824907
6     -0.295329  0.025190      15        1  3.669807, topic_info=          Term          Freq         Total Category  logprob  loglift
33        citi   7176.000000   7176.000000  Default  30.0000  30.0000
472     school   4928.000000   4928.000000  Default  29.0000  29.0000
253      polic   3894.000000   3894.000000  Default  28.0000  28.0000
24    baltimor  12505.000000  12505.000000  Default  27.0000  27.0000
858   maryland   3937.000000   3937.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
1066     sever    326.870715    678.293026  Topic15  -4.3467   2.5750
787      south    246.224136    445.331371  Topic15  -4.6300   2.7125
702      flood    203.218137    289.772777  Topic15  -4.8219   2.9502
1112     coach    217.379130    371.226026  Topic15  -4.7546   2.7699
732      state    296.293153   2580.130338  Topic15  -4.4449   1.1408

[715 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
591       3  0.271372      abl
591      11  0.727008      abl
1333     15  0.997597  absolut
1391      1  0.997792  academi
3255      2  0.157840   accord
...     ...       ...      ...
159      10  0.040308     year
159      11  0.148357     year
190       5  0.999323  yearold
3183      5  0.996060     york
197      11  0.997615     your

[1037 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 3, 15, 6, 4, 10, 1, 9, 5, 14, 13, 2, 11, 8, 7])

In [86]:
from gensim.models.coherencemodel import CoherenceModel

In [87]:
base_perplexity = tweets_lda.log_perplexity(twit_bag)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=tweets_lda, texts=text_clean, 
                                   dictionary=dictionary, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -9.456484574463452

Coherence Score:  0.3688503994902092
